In [1]:
import os
import vtk
from vmtk import pypes
import traceback
import pickle
import networkx as nx
from auxiliares import read_vtk, save_r_p, read_obj, vtpToObj
from parseObj import calcularMatriz
from vedo import *
import Arbol as modelo

MESH OBJ to VTP

In [4]:
files = os.listdir('../mallasArregladas')
for file in files:
    if file.split(".")[1] == "obj" and (file.split(".")[0] + ".vtp") not in files:
        data = read_obj('../mallasArregladas/'+file)
        file = file.split(".")[0]
        writer = vtk.vtkXMLPolyDataWriter();
        writer.SetFileName('../mallasArregladas/'+file+".vtp");
        writer.SetInputData(data);
        writer.Write();

Obtain centerline

In [4]:
#files = ["ArteryObjAN1-11.obj"]
centerfolder = os.listdir("../centerlines")
for file in files:
    if file.split(".")[1] == "vtp":
        if file.split(".")[0] + '-network.vtp' not in centerfolder:
            script = 'vmtknetworkextraction '
            input_file =  '-ifile ../mallasArregladas/' + file 
            output_file = ' -ofile ../centerlines/' + file.split(".")[0] + '-network.vtp'
            myPype = pypes.PypeRun(script+input_file+output_file)

In [5]:
cfolder = os.listdir('../crossSections')
for file in files:
    ##Load mesh and centerline
    msh = load("../mallasArregladas/"+file)
    centerline = read_vtk("../centerlines/" + file.split(".")[0] + "-network.vtp") 
    if file.split(".")[1] == "obj":
        if file.split(".")[0] + '-sections.obj' not in cfolder:
            try:
                save_r_p(centerline, msh, file)
                print("Calculated: ", file)
            except Exception:
                traceback.print_exc()
                print("problem with: ", file)

KeyboardInterrupt: 

centerline to obj

In [2]:
centerfolder = os.listdir("../centerlines")
for file in centerfolder:
    if file.split(".")[1] == "vtp" and (file.split(".")[0] + '.obj') not in centerfolder:
        print("converting", file)
        vtpToObj(file)

converting ArteryObjAN1-0-network.vtp


create graph

In [16]:
gfolder = os.listdir('../grafos')
l_error = []
for file in files:
    if file.split(".")[1] is not None:
        if file.split(".")[1] == "obj":
            fileObj = open("../centerlines/" +file.split(".")[0] +"-network.obj")
            if file.split(".")[0] + '-grafo.gpickle' not in gfolder:
            #if True:
                try: 
                    grafo = calcularMatriz(fileObj, "../radius/" + file.split(".")[0] + "-radius.npy")
                    print("calculating: ", file)
                    with open("../grafos/" + file.split(".")[0] + '-grafo.gpickle', 'wb') as f:
                        #print(grafo.nodes('radio'))
                        pickle.dump(grafo, f, pickle.HIGHEST_PROTOCOL)
                except Exception:
                    print("problem with: ", file)
                    l_error.append(file)
                    #traceback.print_exc()
#print("problem with files: ", l_error)

problem with:  ArteryObjAN134-3.obj
problem with:  ArteryObjAN138-13.obj
problem with:  ArteryObjAN138-17.obj
problem with:  ArteryObjAN152-8.obj
problem with:  ArteryObjAN153-19.obj
problem with:  ArteryObjAN158-17.obj
problem with:  ArteryObjAN160-19.obj
problem with:  ArteryObjAN164-8.obj
problem with:  ArteryObjAN165-3.obj
problem with:  ArteryObjAN165-7.obj
problem with:  ArteryObjAN166-3.obj
problem with:  ArteryObjAN166-7.obj
problem with:  ArteryObjAN167-11.obj
problem with:  ArteryObjAN167-19.obj
problem with:  ArteryObjAN171-12.obj
problem with:  ArteryObjAN174-8.obj
problem with:  ArteryObjAN174-9.obj
problem with:  ArteryObjAN177-10.obj
problem with:  ArteryObjAN181-11.obj
problem with:  ArteryObjAN181-8.obj
problem with:  ArteryObjAN182-9.obj
problem with:  ArteryObjAN183-1.obj
problem with:  ArteryObjAN19-13.obj
problem with:  ArteryObjAN23-9.obj
problem with:  ArteryObjAN32-10.obj
problem with:  ArteryObjAN32-14.obj
problem with:  ArteryObjAN32-5.obj
problem with:  Arter

create binary tree

In [17]:
t_list = os.listdir('../trees')

for file in files:
    if file.split(".")[0] + '-grafo.gpickle' in gfolder and file.split(".")[1] == 'obj' and (file.split(".")[0] + '_tree.dat') not in t_list:
        print("file", file)
        grafo = pickle.load(open('../grafos/' + file.split(".")[0] + '-grafo.gpickle',  'rb'))
        grafo = grafo.to_undirected()

        nc = 0
        #controlo si tiene ciclos
        if len(nx.cycle_basis(grafo))>0:
            nc = 1
        

        nb = 0
        #controlo si es binario (no hay nodos con mas de 3 edges)
        for nodo in grafo.nodes:
            if len(grafo.edges(nodo))>3:
                nb = 1
                break
        
        if nb ==0 and nc == 0: 
            aRecorrer = []
            numeroNodoInicial = 1
            distancias = nx.floyd_warshall( grafo )

            parMaximo = (-1, -1)
            maxima = -1
        
            for nodoInicial in distancias.keys():
                for nodoFinal in distancias[nodoInicial]:
                    if distancias[nodoInicial][nodoFinal] > maxima:
                        maxima = distancias[nodoInicial] [nodoFinal]
                        parMaximo = (nodoInicial, nodoFinal)

            for nodo in grafo.nodes:
                if distancias[parMaximo[0]][nodo] == int( maxima / 2):
                    numeroNodoInicial = nodo
                    if len(grafo.edges(numeroNodoInicial))>2:
                        numeroNodoInicial = list(grafo.edges(numeroNodoInicial))[0][1]

                    break
        
    
            rad = list(grafo.nodes[numeroNodoInicial]['radio'])
            nodoRaiz = modelo.Node( numeroNodoInicial, radius =  rad )

            for vecino in grafo.neighbors( numeroNodoInicial ):
                if vecino != numeroNodoInicial:
                    aRecorrer.append( (vecino, numeroNodoInicial,nodoRaiz ) )
            
            while len(aRecorrer) != 0:
                nodoAAgregar, numeroNodoPadre,nodoPadre = aRecorrer.pop(0)
                radius = list(grafo.nodes[nodoAAgregar]['radio'])
   
                nodoActual = modelo.Node( nodoAAgregar, radius =  radius)
                nodoPadre.agregarHijo( nodoActual )
                for vecino in grafo.neighbors( nodoAAgregar ):
                    if vecino != numeroNodoPadre:
                        aRecorrer.append( (vecino, nodoAAgregar,nodoActual) )

 
            serial = nodoRaiz.serialize(nodoRaiz)
            #write serialized string to file
            file = open("../Trees/" + file.split(".")[0] +"_tree.dat", "w")
            file.write(serial)
            file.close() 
        else:
            print("error with file: ", file, nb, nc)


file ArteryObjAN1-0.obj
True
True
True
file ArteryObjAN1-0.obj
file ArteryObjAN1-0.vtp
True
False
True
file ArteryObjAN1-11.obj
True
True
False
file ArteryObjAN1-11.vtp
True
False
False
file ArteryObjAN1-12.obj
True
True
False
file ArteryObjAN1-12.vtp
True
False
False
file ArteryObjAN1-15.obj
True
True
False
file ArteryObjAN1-15.vtp
True
False
False
file ArteryObjAN1-16.obj
True
True
False
file ArteryObjAN1-16.vtp
True
False
False
file ArteryObjAN1-17.obj
True
True
False
file ArteryObjAN1-17.vtp
True
False
False
file ArteryObjAN1-19.obj
True
True
False
file ArteryObjAN1-19.vtp
True
False
False
file ArteryObjAN1-2.obj
True
True
False
file ArteryObjAN1-2.vtp
True
False
False
file ArteryObjAN1-4.obj
True
True
False
file ArteryObjAN1-4.vtp
True
False
False
file ArteryObjAN1-5.obj
True
True
False
file ArteryObjAN1-5.vtp
True
False
False
file ArteryObjAN1-6.obj
True
True
False
file ArteryObjAN1-6.vtp
True
False
False
file ArteryObjAN1-7.obj
True
True
False
file ArteryObjAN1-7.vtp
True
False


: 

: 

: 

: 

: 

: 

: 

: 